<a href="https://colab.research.google.com/github/midhunjmes/pres_noun/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spacy


In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install openpyxl

In [32]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

def detect_noun(file_path):
    # Detect file type and read accordingly
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")  # Ensure CSV is read properly
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)  # Ensure Excel is handled
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []
    for col in df.columns:
        for value in df[col].astype(str).head(3):  # Check first 3 values per column
            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:  # Identify Nouns & Proper Nouns
                    if col not in sensitive:
                        sensitive.append(col)
    return sensitive




In [34]:
import pandas as pd
import re

def remove_similar_columns(file_path,sensitive_columns):
    """
    Reads a CSV or Excel file, removes column names similar to 'description', 'remarks', etc.,
    and returns a filtered list of sensitive columns.
    """
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()

    # Filter out columns that match the keywords
    filtered_columns = [col for col in sensitive_columns if col in all_columns and not any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]

    return filtered_columns




In [6]:
pip install presidio_analyzer presidio_anonymizer flair regex

In [7]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
from presidio_analyzer import AnalyzerEngine,PatternRecognizer,Pattern,RecognizerResult
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import re
from flair.data import Sentence
from flair.nn import Classifier

In [9]:
ner_tagger = Classifier.load("flair/ner-english-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2025-03-19 10:32:29,197 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [10]:
import pandas as pd
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)



provider = NlpEngineProvider(nlp_configuration=nlp_config)
nlp_engine = provider.create_engine()

analyzer=AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer=AnonymizerEngine()
analyzer.registry.add_recognizer(street_recognizer)

def detect_locations_with_flair(text):
    sentence = Sentence(text)
    ner_tagger.predict(sentence)

    location_entities = []
    for entity in sentence.get_spans("ner"):
        if entity.tag in ["LOC","ORG", 'PER']:
          entity_type = "LOCATION" if entity.tag == "LOC" else "ORGANIZATION" if entity.tag == "ORG" else "PERSON"
          location_entities.append(RecognizerResult(
                entity_type=entity_type,
                start=entity.start_position,
                end=entity.end_position,
                score=entity.score
            ))
    return location_entities

def analyze_and_anonymize(text):
    presidio_results = analyzer.analyze(
        text=text,
        entities=["PERSON","EMAIL_ADDRESS", "PHONE_NUMBER", "STREET_ADDRESS"],
        language="en",
    )

    flair_results = detect_locations_with_flair(text)
    all_results =  flair_results + presidio_results

    anonymized_text = anonymizer.anonymize(text, all_results).text
    return anonymized_text



In [36]:
file_path="test1.xlsx"
sensitive=detect_noun(file_path)
remove_similar_columns(file_path,sensitive)

['Cluster Name', 'Account Name', 'Onboarding Trainings', 'Client Onboarding']

In [38]:
import openpyxl
import json
mapping={}
def excel_an(input_file, output_file):
    wb = openpyxl.load_workbook(input_file)
    sensitive_old =detect_noun(input_file)
    sensitive=remove_similar_columns(file_path,sensitive_old)
    desc = list(set(sensitive_old) - set(sensitive))
    #______________________________________________________________________________________________
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        headers = [cell.value for cell in sheet[1]]  # Get column names

        column_counters = {header: 1 for header in headers if header in sensitive}  # Initialize counters for each column

        for row in sheet.iter_rows(min_row=2):
            for idx, cell in enumerate(row):
                column_name = headers[idx]
                if column_name in sensitive:
                    if cell.value and isinstance(cell.value, str):
                        original_value = cell.value
                        anonymized_value = f"{column_name}{column_counters[column_name]}"
                        cell.value = anonymized_value
                        column_counters[column_name] += 1  # Increment counter
                        mapping[anonymized_value] = original_value
    #______________________________________________________________________________________________
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]
        headers = [cell.value for cell in sheet[1]]  # Get column names
        for header in headers:
            if header in desc:
                for row in sheet.iter_rows(min_row=2):
                    for cell in row:
                        if isinstance(cell.value, str):
                            cell.value = analyze_and_anonymize(cell.value)



    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

    wb.save(output_file)
    print(f"Anonymized file saved as {output_file}")

In [39]:
excel_an("test1.xlsx","pres_an_2.xlsx")

Anonymized file saved as pres_an_2.xlsx


In [40]:
import pandas as pd

mapping = {}

def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    sensitive = remove_similar_columns(input_file, sensitive_old)
    desc=list(set(sensitive_old) - set(sensitive))
    column_counters = {col: 1 for col in df.columns if col in sensitive_old}

    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                anonymized_value = f"{col}{column_counters[col]}"
                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
                column_counters[col] += 1  # Increment counter
            else:
                new_values.append(val)
        df[col] = new_values
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                df.at[idx, col] = analyze_and_anonymize(val)

    df.to_csv(output_file, index=False)
    print(f"Anonymized file saved as {output_file}")

    # Save mapping as JSON
    import json
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)



input_csv = "test_2.csv"
output_csv = "anonymized_file.csv"
csv_an(input_csv, output_csv)


Anonymized file saved as anonymized_file.csv


In [41]:
input_csv = "test_2.csv"
output_csv = "anonymized_file.csv"
csv_an(input_csv, output_csv)

Anonymized file saved as anonymized_file.csv
